In [12]:
!nvidia-smi

Thu May 23 10:36:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   24C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import os
HOME = os.getcwd()
print(HOME)

/home/ec2-user/SageMaker


In [14]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (16 CPUs, 62.0 GB RAM, 87.3/135.0 GB disk)


In [15]:
import ultralytics
import datetime
import shutil
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from sklearn.model_selection import KFold
from ultralytics import YOLO

In [16]:
dataset_path = Path("/home/ec2-user/SageMaker/aws_data/total_dataset")  # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*.txt"))  # all data in 'labels'
yaml_file = "/home/ec2-user/SageMaker/dataset_aws.yaml"  # your data YAML with data directories and names dictionary

In [17]:
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]

cls_idx = list(range(len(classes)))

In [18]:
indx = [l.stem for l in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame(0, columns=cls_idx, index=indx)

In [19]:
for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for l in lines:
        try:
            # classes for YOLO label uses integer at first position of each line
            cls_id = int(float(l.split()[0]))
            lbl_counter[cls_id] += 1
        except ValueError as e:
            print(f"Error processing line '{l.strip()}' in file '{label}': {e}")

    for cls_id, count in lbl_counter.items():
        labels_df.at[label.stem, cls_id] = count

labels_df = labels_df.fillna(0.0)

print(labels_df)

                        0   1   2   3   4   5   6   7   8   9   ...  32  33  \
2024032609093515_aug2    0   4   0   0   0   0   0   0   0   0  ...   0   0   
2024032609093515_aug3    0   4   0   0   0   0   0   0   0   0  ...   0   0   
2024032609094166_aug18   0   4   0   0   0   0   0   0   0   0  ...   0   0   
2024032609094166_aug6    0   4   0   0   0   0   0   0   0   0  ...   0   0   
2024032609102106_aug15   0   5   0   0   0   0   0   0   0   0  ...   0   0   
...                     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   
S_8544.42-2090_20_424    0   0   0   0   0   0   0   0   0   0  ...   0   0   
S_8544.42-2090_20_443    0   0   0   0   0   0   0   0   0   0  ...   0   0   
S_8544.42-2090_20_455    0   0   0   0   0   0   0   0   0   0  ...   0   0   
S_8544.42-2090_20_462    0   0   0   0   0   0   0   0   0   0  ...   0   0   
S_8544.42-2090_20_485    0   0   0   0   0   0   0   0   0   0  ...   0   0   

                        34  35  36  37  38  39  40 

In [20]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [21]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{idx}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{idx}"].loc[labels_df.iloc[val].index] = "val"

In [22]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

In [23]:
# 이미지 파일 수집
supported_extensions = [".jpg", ".jpeg", ".png"]
images = []

# 디버깅을 위해 경로를 출력합니다.
print(f"Looking for images in: {dataset_path}")

Looking for images in: /home/ec2-user/SageMaker/aws_data/total_dataset


In [24]:
supported_extensions = [".jpg", ".jpeg", ".png"]
images = []

for ext in supported_extensions:
    found_images = list(dataset_path.rglob(f"*{ext}")) + list(dataset_path.rglob(f"*{ext.upper()}"))
    images.extend(found_images)
    print(f"Found {len(found_images)} images with extension {ext}")

    # 각 파일 경로와 확장자 확인
    for img in found_images[:1000]:  # 예시로 처음 10개 파일만 출력
        print(img)

# 최종 이미지 개수를 출력합니다.
print(f"Total images found: {len(images)}")

Found 843 images with extension .jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240221_104900_140005_SUPERVISOR_A_aug5.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240228_145815_140005_SUPERVISOR_A_aug9.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240304_105829_140005_0_A_aug1.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240228_153038_140005_SUPERVISOR_A_aug15.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240221_100834_140005_SUPERVISOR_A_aug10.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240221_094140_140005_0_A_aug13.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240221_094108_140005_0_A.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240226_101313_140005_SUPERVISOR_A_aug0.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240221_094739_140005_0_A_aug18.jpg
/home/ec2-user/SageMaker/aws_data/total_dataset/BAGGAGE_20240306_105417_140005_SUPERVISOR_A

In [25]:
# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )
        
        
# 디버깅을 위해 리스트의 내용을 출력
print(f"Total images: {len(images)}")
print(f"Total labels: {len(labels)}")

Total images: 5455
Total labels: 5455


In [26]:
for image in images:
    if image.stem in folds_df.index:
        for split in folds_df.columns:
            k_split = folds_df.loc[image.stem, split]
            if isinstance(k_split, str) and pd.notna(k_split):
                # Destination directory
                img_to_path = save_path / split / k_split / "images"
                lbl_to_path = save_path / split / k_split / "labels"

                # Copy image and label files to new directory (SamefileError if file already exists)
                shutil.copy(image, img_to_path / image.name)
                corresponding_label = dataset_path / f"{image.stem}.txt"
                if corresponding_label.exists():
                    shutil.copy(corresponding_label, lbl_to_path / corresponding_label.name)

In [27]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

In [32]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")
model.info()

results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
# Run inference with the YOLOv8n model on the 'bus.jpg' image
results = model("path/to/bus.jpg")

weights_path = "/home/ec2-user/SageMaker/runs/detect/train5/weights/best.pt"
model = YOLO(weights_path, task="detect")

In [33]:
results = {}

# Define your additional arguments here
batch = 16
project = "kfold_demo"
epochs = 300
lr = 0.001

for k in range(ksplit):
    dataset_yaml = ds_yamls[k]
    model.train(data=dataset_yaml, epochs=epochs, batch=batch, project=project, lr0=lr)  
    results[k] = model.metrics

New https://pypi.org/project/ultralytics/8.2.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/home/ec2-user/SageMaker/runs/detect/train5/weights/best.pt, data=/home/ec2-user/SageMaker/aws_data/total_dataset/2024-05-23_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=300, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_demo, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=

      1/300      2.85G     0.6977     0.6719      0.931        164        640:   3%|▎         | 8/273 [00:06<03:30,  1.26it/s]


RuntimeError: DataLoader worker (pid(s) 4104, 4108, 4109, 4110, 4117, 4122, 4123, 4124) exited unexpectedly

In [ ]:
# 저장